In [ ]:
"""
Train an ensemble of noisy Hybrid VQC models (5 experts, same T2 noise type),
each with a different random seed, for binary face verification.

Assumes:
  - A frozen ResNet-18 backbone checkpoint exists at:
        outputs/resnet18_backbone_only.pt
    (or outputs/resnet18_finetuned.pt as a fallback)
  - Training and validation images are in:
        data/train   (subfolders: negative, positive)
        data/val     (subfolders: negative, positive)
"""

import os, random, copy, time
from pathlib import Path
from typing import List, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from tqdm.auto import tqdm

import pennylane as qml
from pennylane import numpy as pnp

# ----------------- Paths & config -----------------
ROOT = Path(".")
BACKBONE_CKPT_1 = ROOT / "outputs" / "resnet18_backbone_only.pt"
BACKBONE_CKPT_2 = ROOT / "outputs" / "resnet18_finetuned.pt"

DATA_ROOT = ROOT / "_bin_dataset"
TRAIN_ROOT = DATA_ROOT / "train"
VAL_ROOT   = DATA_ROOT / "val"

NOISE_CKPT_DIR = ROOT / "outputs" / "noise_models"
NOISE_CKPT_DIR.mkdir(parents=True, exist_ok=True)

# 5 experts for one noise type
N_T2_EXPERTS = 5

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Global (base) seed – we’ll offset per expert
BASE_SEED = 123

# ImageNet stats
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

# Data transforms
train_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

val_tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

tfm = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

print("Train dir:", TRAIN_ROOT.resolve())
print("Val dir  :", VAL_ROOT.resolve())
print("Noise models will be saved to:", NOISE_CKPT_DIR.resolve())


c:\ECE733\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda
Train dir: C:\ECE733\Final\_bin_dataset\train
Val dir  : C:\ECE733\Final\_bin_dataset\val
Noise models will be saved to: C:\ECE733\Final\outputs\noise_models


In [2]:
#############################################
# 1. Frozen ResNet-18 backbone + 512→4 projector
#############################################

def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def load_frozen_backbone() -> Tuple[nn.Module, int]:
    """
    Load the frozen ResNet-18 backbone.
    Tries outputs/resnet18_backbone_only.pt, then outputs/resnet18_finetuned.pt.
    """
    if BACKBONE_CKPT_1.exists():
        ckpt_backbone = BACKBONE_CKPT_1
    elif BACKBONE_CKPT_2.exists():
        ckpt_backbone = BACKBONE_CKPT_2
    else:
        raise FileNotFoundError(
            "Backbone weights not found.\n"
            "Expected one of:\n"
            f"  {BACKBONE_CKPT_1}\n"
            f"  {BACKBONE_CKPT_2}"
        )

    backbone = models.resnet18(weights=None)
    state = torch.load(ckpt_backbone, map_location="cpu")
    if isinstance(state, dict) and "state_dict" in state:
        state = state["state_dict"]
    backbone.load_state_dict(state, strict=False)  # ignore fc mismatch

    in_feats = backbone.fc.in_features
    backbone.fc = nn.Identity()

    backbone.to(device)
    backbone.eval()
    for p in backbone.parameters():
        p.requires_grad_(False)

    print("Loaded backbone from:", ckpt_backbone)
    print("Backbone output dim:", in_feats)
    return backbone, in_feats

BACKBONE, BACKBONE_OUT = load_frozen_backbone()

class L512to4(nn.Module):
    """
    512-d CNN feature -> 4-d embedding for quantum circuit input.
    """
    def __init__(self, in_dim=512, hidden_dim=4):
        super().__init__()
        self.fc = nn.Linear(in_dim, hidden_dim)
        self.act = nn.Tanh()  # keep within [-1, 1] range

    def forward(self, z: torch.Tensor) -> torch.Tensor:
        return self.act(self.fc(z))  # [B, 4]


Loaded backbone from: outputs\resnet18_backbone_only.pt
Backbone output dim: 512


In [ ]:
#############################################
# 2. Quantum circuit definition with noise
#############################################

n_qubits = 4
n_layers = 6

# Default noise strengths
T1_gamma_default = 0.01
T2_gamma_default = 0.02
epsilon_default  = 0.03  # only used for rotation noise

# Mixed-state device to support noisy channels
qdev = qml.device("default.mixed", wires=n_qubits)

def entangle_ladder():
    """
    Simple ladder entangling pattern on 4 qubits.
    """
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])

@qml.qnode(qdev, interface="torch")
def quantum_block(x, weights,
                  noise_type="none",
                  gamma_T1=0.0,
                  gamma_T2=0.0,
                  epsilon=0.0):
    """
    x:        Tensor of shape [4] (classical embedding).
    weights:  Tensor [n_layers, n_qubits].
    noise_type: "none", "T1", "T2", "rotation"
    gamma_T1: amplitude damping parameter
    gamma_T2: dephasing parameter
    epsilon:  over-rotation magnitude (for 'rotation' noise)
    """

    def apply_noise():
        # Inject noise after each layer
        if noise_type == "T1":
            for q in range(n_qubits):
                qml.AmplitudeDamping(gamma_T1, wires=q)
        elif noise_type == "T2":
            for q in range(n_qubits):
                qml.PhaseDamping(gamma_T2, wires=q)
        elif noise_type == "T12" or noise_type == "T12r":
            for q in range(n_qubits):
                qml.AmplitudeDamping(gamma_T1, wires=q)
                qml.PhaseDamping(gamma_T2, wires=q)
        # rotation noise is handled in the gate angles

    # ----- 1) Data encoding -----
    for q in range(n_qubits):
        qml.Hadamard(wires=q)
        qml.RY(np.pi * x[q] / 2.0, wires=q)

    apply_noise()

    # ----- 2) Variational layers -----
    for l in range(n_layers):
        # Single-qubit rotations
        for q in range(n_qubits):
            if noise_type == "rotation" or noise_type == "T12r":
                qml.RY(weights[l, q] + epsilon, wires=q)
            else:
                qml.RY(weights[l, q], wires=q)

        apply_noise()
        entangle_ladder()

    # ----- 3) Measurement -----
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]


In [ ]:
#############################################
# 3. Quantum layer, classifier head, Hybrid model
#############################################

# Helper: entanglement patterns
def make_entangler(kind: str):
    if kind == "ladder":
        return [(1, 2), (0, 1), (2, 3)]
    if kind == "ring":
        # simple ring across 4 qubits
        return [(0, 1), (1, 2), (2, 3), (3, 0)]
    pairs = [(0, 1), (1, 2), (2, 3), (0, 2), (1, 3)]
    random.shuffle(pairs)
    return pairs[:4]


class QuantumLayer(nn.Module):
    """
    Variational quantum layer with noise.

    Compared to the original version, we now allow:
      - variable depth (number of layers) per expert
      - variable entanglement pattern per expert

    `depth` / `pattern` can be passed explicitly, otherwise we fall back to
    a small random choice (like in the clean ensemble notebook).
    """
    def __init__(
        self,
        noise_type="none",
        T1=0.0,
        T2=0.0,
        epsilon=0.0,
        depth=None,
        pairs=None,
        pattern=None,
        shots=None,
    ):
        super().__init__()

        # ---- circuit structure (varies per expert) ----
        self.depth = int(depth) if depth is not None else random.choice([5, 6])

        if pairs is not None:
            self.pairs = list(pairs)
        else:
            pat = pattern or random.choice(["ladder", "ring", "rand"])
            self.pairs = make_entangler(pat)

        # learnable parameters: [depth, n_qubits]
        self.weights = nn.Parameter(0.01 * torch.randn(self.depth, n_qubits))

        # ---- noise configuration ----
        self.noise_type = noise_type
        self.gamma_T1 = T1
        self.gamma_T2 = T2
        self.epsilon = epsilon

        # each expert gets its own mixed-state device / QNode
        self.dev = qml.device("default.mixed", wires=n_qubits, shots=shots)

        def circuit(x, w):
            # capture current config into local vars for the QNode
            noise_type = self.noise_type
            gamma_T1 = self.gamma_T1
            gamma_T2 = self.gamma_T2
            epsilon = self.epsilon
            depth = self.depth
            pairs = self.pairs

            def apply_noise():
                if noise_type == "T1":
                    for q in range(n_qubits):
                        qml.AmplitudeDamping(gamma_T1, wires=q)
                elif noise_type == "T2":
                    for q in range(n_qubits):
                        qml.PhaseDamping(gamma_T2, wires=q)
                elif noise_type == "T12" or noise_type == "T12r":
                    for q in range(n_qubits):
                        qml.AmplitudeDamping(gamma_T1, wires=q)
                        qml.PhaseDamping(gamma_T2, wires=q)
                # rotation noise is handled in the gate angles

            # ----- 1) Data encoding -----
            for q in range(n_qubits):
                qml.Hadamard(wires=q)
                qml.RY(np.pi * x[q] / 2.0, wires=q)

            apply_noise()

            # ----- 2) Variational layers -----
            for l in range(depth):
                # Single-qubit rotations
                for q in range(n_qubits):
                    if noise_type == "rotation" or noise_type == "T12r":
                        qml.RY(w[l, q] + epsilon, wires=q)
                    else:
                        qml.RY(w[l, q], wires=q)

                apply_noise()

                # entanglement pattern chosen per expert
                for a, b in pairs:
                    qml.CNOT(wires=[a, b])

            # ----- 3) Measurement -----
            return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

        self.qnode = qml.QNode(circuit, self.dev, interface="torch")

    def forward(self, x4_batch: torch.Tensor) -> torch.Tensor:
        outs = []
        for i in range(x4_batch.shape[0]):
            y = self.qnode(x4_batch[i], self.weights)
            if not isinstance(y, torch.Tensor):
                y = torch.stack(y)
            outs.append(y)
        return torch.stack(outs, dim=0).to(torch.float32)


class L4to2(nn.Module):
    def __init__(self, in_dim=4, hidden_dim=8, num_classes=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, num_classes),
        )

    def forward(self, x):
        return self.net(x)


class HybridModel(nn.Module):
    def __init__(self, backbone, proj, q_layer, head):
        super().__init__()
        self.backbone = backbone
        self.proj = proj
        self.q_layer = q_layer
        self.head = head

    def forward(self, imgs: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():  # backbone frozen
            z512 = self.backbone(imgs)  # [B, 512]

        x4 = self.proj(z512)            # [B, 4]
        zq = self.q_layer(x4)           # [B, 4]
        logits = self.head(zq)          # [B, 2]
        return logits


In [5]:
#############################################
# 4. Datasets and Dataloaders
#############################################

def make_dataloaders(batch_size: int = 16, num_workers: int = 0):
    if not TRAIN_ROOT.exists():
        raise FileNotFoundError(f"Train dir not found: {TRAIN_ROOT}")
    if not VAL_ROOT.exists():
        raise FileNotFoundError(f"Val dir not found: {VAL_ROOT}")

    train_ds = datasets.ImageFolder(TRAIN_ROOT, transform=train_tfm)
    val_ds   = datasets.ImageFolder(VAL_ROOT,   transform=val_tfm)

    print("Train classes:", train_ds.classes)
    print("Val classes  :", val_ds.classes)
    assert train_ds.classes == val_ds.classes, "Train and val classes must match."

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_loader, val_loader, train_ds.classes

train_loader, val_loader, CLASS_NAMES = make_dataloaders(batch_size=16, num_workers=0)
print("Number of training samples:", len(train_loader.dataset))
print("Number of validation samples:", len(val_loader.dataset))


Train classes: ['Negative', 'Positive']
Val classes  : ['Negative', 'Positive']
Number of training samples: 480
Number of validation samples: 120


In [ ]:
#############################################
# 5. Generic noise-expert utilities
#############################################

T1_gamma_default = 0.02   # amplitude damping strength
T2_gamma_default = 0.049   # dephasing strength
epsilon_default  = 0.03   # over-rotation magnitude

# ------------------------------------------------------------------
# Ensemble structure configuration: depth + entanglement per expert
# ------------------------------------------------------------------

# Fixed small menu of structures (same for all noise types).
# With 5 experts, each one gets a unique pattern pair.
STRUCT_CONFIGS = [
    {"depth": 6, "pattern": "ladder"},
    {"depth": 6, "pattern": "ring"},
    {"depth": 6, "pattern": "ladder"},
    {"depth": 6, "pattern": "ring"},
    {"depth": 6, "pattern": "rand"},
]


def get_structure_for_expert(expert_id: int):
    cfg = STRUCT_CONFIGS[expert_id % len(STRUCT_CONFIGS)]
    return cfg["depth"], cfg["pattern"]


def make_noise_expert_model(noise_type="T2", expert_id: int = 0) -> HybridModel:
    """
    Build a fresh Hybrid model configured for a given noise_type and *structure*.

    noise_type ∈ {"none", "T1", "T2", "rotation"}.
    The circuit structure (depth + entanglement pattern) is chosen
    deterministically from expert_id via get_structure_for_expert().
    """
    backbone = copy.deepcopy(BACKBONE)
    proj = L512to4(in_dim=BACKBONE_OUT, hidden_dim=4)

    depth, pattern = get_structure_for_expert(expert_id)
    common_kwargs = {"depth": depth, "pattern": pattern}

    if noise_type == "T1":
        q_layer = QuantumLayer(
            noise_type="T1",
            T1=T1_gamma_default,
            T2=0.0,
            epsilon=0.0,
            **common_kwargs,
        )
    elif noise_type == "T2":
        q_layer = QuantumLayer(
            noise_type="T2",
            T1=0.0,
            T2=T2_gamma_default,
            epsilon=0.0,
            **common_kwargs,
        )
    elif noise_type == "T12":
        q_layer = QuantumLayer(
            noise_type="T12",
            T1=T1_gamma_default,
            T2=T2_gamma_default,
            epsilon=0.0,
            **common_kwargs,
        )
    elif noise_type == "T12r":
        q_layer = QuantumLayer(
            noise_type="T12r",
            T1=T1_gamma_default,
            T2=T2_gamma_default,
            epsilon=epsilon_default,
            **common_kwargs,
        )
    elif noise_type == "rotation":
        q_layer = QuantumLayer(
            noise_type="rotation",
            T1=0.0,
            T2=0.0,
            epsilon=epsilon_default,
            **common_kwargs,
        )
    else:  # "none" or anything else
        q_layer = QuantumLayer(
            noise_type="none",
            T1=0.0,
            T2=0.0,
            epsilon=0.0,
            **common_kwargs,
        )

    head = L4to2()
    model = HybridModel(backbone, proj, q_layer, head).to(device)
    return model


def accuracy_from_logits(logits: torch.Tensor, labels: torch.Tensor) -> float:
    preds = logits.argmax(dim=1)
    correct = (preds == labels).float().sum().item()
    total = labels.size(0)
    return correct / total if total > 0 else 0.0

def train_one_epoch(model: nn.Module,
                    loader: DataLoader,
                    optimizer: torch.optim.Optimizer,
                    criterion: nn.Module):
    model.train()
    total_loss = 0.0
    total_acc = 0.0
    total_count = 0

    for imgs, labels in tqdm(loader, desc="Train", leave=False):
        imgs = imgs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = model(imgs)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        batch_size = labels.size(0)
        total_loss += loss.item() * batch_size
        total_acc  += accuracy_from_logits(logits, labels) * batch_size
        total_count += batch_size

    return total_loss / total_count, total_acc / total_count


@torch.no_grad()
def eval_one_epoch(model: nn.Module,
                   loader: DataLoader,
                   criterion: nn.Module):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0
    total_count = 0

    for imgs, labels in tqdm(loader, desc="Val", leave=False):
        imgs = imgs.to(device)
        labels = labels.to(device)

        logits = model(imgs)
        loss = criterion(logits, labels)

        batch_size = labels.size(0)
        total_loss += loss.item() * batch_size
        total_acc  += accuracy_from_logits(logits, labels) * batch_size
        total_count += batch_size

    return total_loss / total_count, total_acc / total_count


def train_noise_expert(
    expert_id: int,
    noise_type: str,
    ensemble_tag: str,
    num_epochs: int = 10,
    lr: float = 1e-3,
    weight_decay: float = 0.0,
    batch_size: int = 16,
) -> Path:
    """
    Train a single noise expert for a given noise_type and ensemble_tag.

    Examples:
      noise_type="T1", ensemble_tag="T1"     -> hybrid_noise_T1_expert{expert_id}.pt
      noise_type="rotation", ensemble_tag="rot"
        -> hybrid_noise_rot_expert{expert_id}.pt
      noise_type="T2", ensemble_tag="T2"     -> hybrid_noise_T2_expert{expert_id}.pt
    """
    # Different seed per expert & noise_type to diversify training data order.
    seed = BASE_SEED + 111 * expert_id + hash(noise_type) % 1000
    set_global_seed(seed)
    print(f"\n=== Training {noise_type} expert {expert_id} "
          f"(ensemble={ensemble_tag}, seed={seed}) ===")

    # Recreate loaders so shuffling uses this seed
    train_loader, val_loader, _ = make_dataloaders(batch_size=batch_size, num_workers=0)

    # *** key change: structure (depth + entanglement) is tied to expert_id ***
    model = make_noise_expert_model(noise_type=noise_type, expert_id=expert_id)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(
        list(model.proj.parameters()) +
        list(model.q_layer.parameters()) +
        list(model.head.parameters()),
        lr=lr,
        weight_decay=weight_decay,
    )

    best_val_acc = 0.0
    ckpt_name = f"hybrid_noise_{ensemble_tag}_expert{expert_id}.pt"
    best_ckpt_path = NOISE_CKPT_DIR / ckpt_name

    for epoch in range(1, num_epochs + 1):
        print(f"\n[{ensemble_tag} expert {expert_id}] Epoch {epoch}/{num_epochs}")
        start_time = time.time()

        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = eval_one_epoch(model, val_loader, criterion)

        elapsed = time.time() - start_time
        print(f"[{ensemble_tag} expert {expert_id}] "
              f"Train loss={train_loss:.4f}, acc={train_acc:.4f} | "
              f"Val loss={val_loss:.4f}, acc={val_acc:.4f} | "
              f"time={elapsed:.1f}s")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_ckpt_path)
            print(f"  -> New best model saved to {best_ckpt_path} (val_acc={best_val_acc:.4f})")

    print(f"\nFinished training {ensemble_tag} expert {expert_id}. "
          f"Best val_acc={best_val_acc:.4f}")
    print(f"Best checkpoint: {best_ckpt_path}")
    return best_ckpt_path


In [ ]:
# #############################################
# # 6. Train the full T2-noise ensemble (5 experts)
# #############################################

# # Hyperparameters for all experts (you can tune these)
# NUM_EPOCHS   = 6
# LR           = 1e-3
# WEIGHT_DECAY = 0.0
# BATCH_SIZE   = 16

# all_ckpts = []
# for expert_id in range(N_T2_EXPERTS):
#     ckpt_path = train_noise_expert(
#         expert_id   = expert_id,
#         noise_type  = "T2",
#         ensemble_tag= "T2",
#         num_epochs  = NUM_EPOCHS,
#         lr          = LR,
#         weight_decay= WEIGHT_DECAY,
#         batch_size  = BATCH_SIZE,
#     )
#     all_ckpts.append(ckpt_path)

# print("\nAll expert checkpoints:")
# for p in all_ckpts:
#     print("  ", p)


In [8]:
# #############################################
# # 6. Train T1 and rotation ensembles (5 experts each)
# #############################################

# N_EXPERTS = 5
# NUM_EPOCHS = 6
# LR = 1e-3
# WEIGHT_DECAY = 0.0
# BATCH_SIZE = 16

# # ---------- T1 ensemble ----------
# print("\n###########################")
# print("# Training T1 ensemble    #")
# print("###########################")

# #############################################
# # Train T1-noise ensemble
# #############################################
# all_T1_ckpts = []
# for expert_id in range(N_EXPERTS):
#     ckpt_path = train_noise_expert(
#         expert_id    = expert_id,
#         noise_type   = "T1",
#         ensemble_tag = "T1",   # filenames: hybrid_noise_T1_expertX.pt
#         num_epochs   = NUM_EPOCHS,
#         lr           = LR,
#         weight_decay = WEIGHT_DECAY,
#         batch_size   = BATCH_SIZE,
#     )
#     all_T1_ckpts.append(ckpt_path)



# # ---------- rotation ensemble ----------
# print("\n###########################")
# print("# Training rotation ensemble")
# print("###########################")

# #############################################
# # Train rotation-noise ensemble
# #############################################
# all_rot_ckpts = []
# for expert_id in range(N_EXPERTS):
#     ckpt_path = train_noise_expert(
#         expert_id    = expert_id,
#         noise_type   = "rotation",
#         ensemble_tag = "rot",   # filenames: hybrid_noise_rot_expertX.pt
#         num_epochs   = NUM_EPOCHS,
#         lr           = LR,
#         weight_decay = WEIGHT_DECAY,
#         batch_size   = BATCH_SIZE,
#     )
#     all_rot_ckpts.append(ckpt_path)


# print("\nRotation expert checkpoints:")
# for p in all_rot_ckpts:
#     print("  ", p)


In [ ]:
#############################################
# 6. Train the full T12-noise ensemble (5 experts)
#############################################

# Hyperparameters for all experts (you can tune these)
NUM_EPOCHS   = 6
LR           = 1e-3
WEIGHT_DECAY = 0.0
BATCH_SIZE   = 16

all_ckpts = []
for expert_id in range(N_T2_EXPERTS):
    ckpt_path = train_noise_expert(
        expert_id   = expert_id,
        noise_type  = "T12",
        ensemble_tag= "T12",
        num_epochs  = NUM_EPOCHS,
        lr          = LR,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
    )
    all_ckpts.append(ckpt_path)

print("\nAll expert checkpoints:")
for p in all_ckpts:
    print("  ", p)



=== Training T12 expert 0 (ensemble=T12, seed=186) ===
Train classes: ['Negative', 'Positive']
Val classes  : ['Negative', 'Positive']

[T12 expert 0] Epoch 1/6


[T12 expert 0] Train loss=0.6359, acc=0.5000 | Val loss=0.5888, acc=0.5000 | time=110.2s
  -> New best model saved to outputs\noise_models\hybrid_noise_T12_expert0.pt (val_acc=0.5000)

[T12 expert 0] Epoch 2/6


[T12 expert 0] Train loss=0.5500, acc=0.5000 | Val loss=0.5066, acc=0.5000 | time=108.3s

[T12 expert 0] Epoch 3/6


[T12 expert 0] Train loss=0.4720, acc=0.7104 | Val loss=0.4230, acc=0.9667 | time=104.7s
  -> New best model saved to outputs\noise_models\hybrid_noise_T12_expert0.pt (val_acc=0.9667)

[T12 expert 0] Epoch 4/6


[T12 expert 0] Train loss=0.3790, acc=0.9833 | Val loss=0.3415, acc=0.9750 | time=108.4s
  -> New best model saved to outputs\noise_models\hybrid_noise_T12_expert0.pt (val_acc=0.9750)

[T12 expert 0] Epoch 5/6


...

[T12 expert 4] Train loss=0.5704, acc=0.5000 | Val loss=0.5504, acc=0.5000 | time=122.6s

[T12 expert 4] Epoch 3/6


[T12 expert 4] Train loss=0.5312, acc=0.5000 | Val loss=0.5132, acc=0.5000 | time=119.4s

[T12 expert 4] Epoch 4/6


[T12 expert 4] Train loss=0.4948, acc=0.5000 | Val loss=0.4801, acc=0.5000 | time=120.5s

[T12 expert 4] Epoch 5/6


[T12 expert 4] Train loss=0.4605, acc=0.5000 | Val loss=0.4502, acc=0.5000 | time=117.6s

[T12 expert 4] Epoch 6/6


[T12 expert 4] Train loss=0.4362, acc=0.5000 | Val loss=0.4281, acc=0.5000 | time=116.1s

Finished training T12 expert 4. Best val_acc=0.5000
Best checkpoint: outputs\noise_models\hybrid_noise_T12_expert4.pt

All expert checkpoints:
   outputs\noise_models\hybrid_noise_T12_expert0.pt
   outputs\noise_models\hybrid_noise_T12_expert1.pt
   outputs\noise_models\hybrid_noise_T12_expert2.pt
   outputs\noise_models\hybrid_noise_T12_expert3.pt
   outputs\noise_models\hybrid_noise_T12_expert4.pt


In [ ]:
#############################################
# 6. Train the full T2-noise ensemble (5 experts)
#############################################

# Hyperparameters for all experts (you can tune these)
NUM_EPOCHS   = 6
LR           = 1e-3
WEIGHT_DECAY = 0.0
BATCH_SIZE   = 16

all_ckpts = []
for expert_id in range(N_T2_EXPERTS):
    ckpt_path = train_noise_expert(
        expert_id   = expert_id,
        noise_type  = "T12r",
        ensemble_tag= "T12r",
        num_epochs  = NUM_EPOCHS,
        lr          = LR,
        weight_decay= WEIGHT_DECAY,
        batch_size  = BATCH_SIZE,
    )
    all_ckpts.append(ckpt_path)

print("\nAll expert checkpoints:")
for p in all_ckpts:
    print("  ", p)


=== Training T12r expert 0 (ensemble=T12r, seed=196) ===
Train classes: ['Negative', 'Positive']
Val classes  : ['Negative', 'Positive']

[T12r expert 0] Epoch 1/6


[T12r expert 0] Train loss=0.5895, acc=0.9167 | Val loss=0.5357, acc=0.9750 | time=105.3s
  -> New best model saved to outputs\noise_models\hybrid_noise_T12r_expert0.pt (val_acc=0.9750)

[T12r expert 0] Epoch 2/6


[T12r expert 0] Train loss=0.4734, acc=0.9792 | Val loss=0.4183, acc=0.9750 | time=103.3s

[T12r expert 0] Epoch 3/6


[T12r expert 0] Train loss=0.3802, acc=0.9688 | Val loss=0.3304, acc=0.9750 | time=113.4s

[T12r expert 0] Epoch 4/6


[T12r expert 0] Train loss=0.2864, acc=0.9875 | Val loss=0.2577, acc=0.9750 | time=104.8s

[T12r expert 0] Epoch 5/6


...

[T12r expert 4] Train loss=0.4803, acc=0.9771 | Val loss=0.4464, acc=0.9750 | time=121.2s
  -> New best model saved to outputs\noise_models\hybrid_noise_T12r_expert4.pt (val_acc=0.9750)

[T12r expert 4] Epoch 3/6


[T12r expert 4] Train loss=0.4069, acc=0.9812 | Val loss=0.3739, acc=0.9750 | time=124.0s

[T12r expert 4] Epoch 4/6


[T12r expert 4] Train loss=0.3302, acc=0.9771 | Val loss=0.2915, acc=0.9750 | time=120.1s

[T12r expert 4] Epoch 5/6


[T12r expert 4] Train loss=0.2685, acc=0.9771 | Val loss=0.2500, acc=0.9750 | time=123.5s

[T12r expert 4] Epoch 6/6


[T12r expert 4] Train loss=0.2086, acc=0.9875 | Val loss=0.1965, acc=0.9750 | time=118.5s

Finished training T12r expert 4. Best val_acc=0.9750
Best checkpoint: outputs\noise_models\hybrid_noise_T12r_expert4.pt

All expert checkpoints:
   outputs\noise_models\hybrid_noise_T12r_expert0.pt
   outputs\noise_models\hybrid_noise_T12r_expert1.pt
   outputs\noise_models\hybrid_noise_T12r_expert2.pt
   outputs\noise_models\hybrid_noise_T12r_expert3.pt
   outputs\noise_models\hybrid_noise_T12r_expert4.pt


In [ ]:
from pathlib import Path
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

CKPT_DIR = Path("outputs/noise_models")
FINAL_ROOT = Path("final_test_full")

# Noise hyperparams
T1_gamma_default = 0.01
T2_gamma_default = 0.02
epsilon_default  = 0.03

# ================================================================
# Inference helpers: build experts with the same structure as train
# ================================================================

def make_noise_expert_model_for_inference(noise_type: str, expert_id: int):
    """
    Build a HybridModel with the right noise configuration *and*
    the same (depth, entanglement pattern) that was used at training
    time for this expert_id.
    """
    # backbone is already frozen globally; use a copy per expert
    backbone = copy.deepcopy(BACKBONE)
    proj = L512to4(in_dim=BACKBONE_OUT, hidden_dim=4)

    depth, pattern = get_structure_for_expert(expert_id)
    common_kwargs = {"depth": depth, "pattern": pattern}

    if noise_type == "T1":
        q_layer = QuantumLayer(
            noise_type="T1",
            T1=T1_gamma_default,
            T2=0.0,
            epsilon=0.0,
            **common_kwargs,
        )
    elif noise_type == "T2":
        q_layer = QuantumLayer(
            noise_type="T2",
            T1=0.0,
            T2=T2_gamma_default,
            epsilon=0.0,
            **common_kwargs,
        )
    elif noise_type == "T12":
        q_layer = QuantumLayer(
            noise_type="T12",
            T1=T1_gamma_default,
            T2=T2_gamma_default,
            epsilon=0.0,
            **common_kwargs,
        )
    elif noise_type == "T12r":
        q_layer = QuantumLayer(
            noise_type="T12r",
            T1=T1_gamma_default,
            T2=T2_gamma_default,
            epsilon=epsilon_default,
            **common_kwargs,
        )
    elif noise_type == "rotation":
        q_layer = QuantumLayer(
            noise_type="rotation",
            T1=0.0,
            T2=0.0,
            epsilon=epsilon_default,
            **common_kwargs,
        )
    else:
        q_layer = QuantumLayer(
            noise_type="none",
            T1=0.0,
            T2=0.0,
            epsilon=0.0,
            **common_kwargs,
        )

    head = L4to2()
    model = HybridModel(backbone, proj, q_layer, head).to(device)
    return model


def load_noise_ensemble(noise_tag: str, noise_type: str):
    """
    noise_tag controls filename pattern:
      - 'T2'  -> hybrid_noise_T2_expert*.pt
      - 'T1'  -> hybrid_noise_T1_expert*.pt
      - 'rot' -> hybrid_noise_rot_expert*.pt
    noise_type is the semantic noise type ("T1", "T2", "rotation").

    We now parse expert_id from the filename so that we can recreate
    the correct circuit structure for each expert.
    """
    pattern = f"hybrid_noise_{noise_tag}_expert*.pt"
    models = []

    for ckpt_path in sorted(CKPT_DIR.glob(pattern)):
        name = ckpt_path.stem
        try:
            expert_id = int(name.split("expert")[-1])
        except ValueError:
            print(f"Warning: could not parse expert id from {name}, skipping.")
            continue

        print(f"Loading {ckpt_path.name} (expert_id={expert_id}, noise_type={noise_type})")
        m = make_noise_expert_model_for_inference(noise_type=noise_type,
                                                  expert_id=expert_id)

        obj = torch.load(ckpt_path, map_location=device)
        # support both plain state_dict and {"model_state": ...}
        if isinstance(obj, dict) and "model_state" in obj:
            state = obj["model_state"]
        else:
            state = obj

        m.load_state_dict(state, strict=False)
        m.eval()
        models.append(m)

    print(f"Loaded {len(models)} experts for noise_tag='{noise_tag}'.")
    return models


# Actually load your ensembles (whatever you trained)
# ensemble_T2  = load_noise_ensemble("T2",  "T2")
# ensemble_T1  = load_noise_ensemble("T1",  "T1")
# ensemble_rot = load_noise_ensemble("rot", "rotation")
ensemble_T12  = load_noise_ensemble("T12",  "T12")
ensemble_T12r  = load_noise_ensemble("T12r",  "T12r")




Loading hybrid_noise_T12_expert0.pt (expert_id=0, noise_type=T12)
Loading hybrid_noise_T12_expert1.pt (expert_id=1, noise_type=T12)
Loading hybrid_noise_T12_expert2.pt (expert_id=2, noise_type=T12)
Loading hybrid_noise_T12_expert3.pt (expert_id=3, noise_type=T12)
Loading hybrid_noise_T12_expert4.pt (expert_id=4, noise_type=T12)
Loaded 5 experts for noise_tag='T12'.
Loading hybrid_noise_T12r_expert0.pt (expert_id=0, noise_type=T12r)
Loading hybrid_noise_T12r_expert1.pt (expert_id=1, noise_type=T12r)
Loading hybrid_noise_T12r_expert2.pt (expert_id=2, noise_type=T12r)
Loading hybrid_noise_T12r_expert3.pt (expert_id=3, noise_type=T12r)
Loading hybrid_noise_T12r_expert4.pt (expert_id=4, noise_type=T12r)
Loaded 5 experts for noise_tag='T12r'.


In [13]:
@torch.no_grad()
def evaluate_ensemble_on_final_test(models_list, name: str = ""):
    if not FINAL_ROOT.exists():
        raise FileNotFoundError(f"final_test directory not found at: {FINAL_ROOT}")

    if len(models_list) == 0:
        print(f"[{name}] No models in ensemble, skipping.")
        return None

    ds = datasets.ImageFolder(FINAL_ROOT, transform=tfm)
    loader = DataLoader(ds, batch_size=64, shuffle=False, num_workers=0)

    class_names = ds.classes
    print(f"\n=== Evaluating ensemble '{name}' on final_test ===")
    print("Classes:", class_names)
    print("Num images:", len(ds))
    print("Num experts in ensemble:", len(models_list))

    all_labels = []
    all_preds_ens = []

    with torch.no_grad():
        for imgs, labels in tqdm(loader, desc=f"final_test [{name}]"):
            imgs = imgs.to(device)
            labels = labels.to(device)

            # Per-expert logits
            logits_list = [m(imgs) for m in models_list]  # list of [B,2]

            # Stack to [E,B,2] then average softmax probabilities
            stacked = torch.stack(logits_list, dim=0)      # [E,B,2]
            probs   = F.softmax(stacked, dim=2)            # [E,B,2]
            avg_probs = probs.mean(dim=0)                  # [B,2]
            ens_preds_batch = avg_probs.argmax(dim=1).cpu()

            all_labels.append(labels.cpu())
            all_preds_ens.append(ens_preds_batch)

    all_labels = torch.cat(all_labels).numpy()
    all_preds_ens = torch.cat(all_preds_ens).numpy()

    acc = float((all_preds_ens == all_labels).mean())
    acc_percent = acc * 100.0

    print(f"\n[{name}] Overall accuracy on final_test: {acc_percent:.2f}%")

    print("\nPer-class accuracy:")
    for idx, cname in enumerate(class_names):
        mask = (all_labels == idx)
        if mask.sum() == 0:
            continue
        cls_acc = float((all_preds_ens[mask] == all_labels[mask]).mean()) * 100.0
        print(f"  Class '{cname}': {cls_acc:.2f}% (n={mask.sum()})")

    return acc_percent


In [15]:
results = {}

# if len(ensemble_T2) > 0:
#     results["T2"] = evaluate_ensemble_on_final_test(ensemble_T2, name="T2")

# if len(ensemble_T1) > 0:
#     results["T1"] = evaluate_ensemble_on_final_test(ensemble_T1, name="T1")

# if len(ensemble_rot) > 0:
#     results["rot"] = evaluate_ensemble_on_final_test(ensemble_rot, name="rotation")

if len(ensemble_T12) > 0:
    results["T12"] = evaluate_ensemble_on_final_test(ensemble_T12, name="T12")

if len(ensemble_T12r) > 0:
    results["T12r"] = evaluate_ensemble_on_final_test(ensemble_T12r, name="T12r")

print("\n\nSummary of final accuracies (percent):")
for k, v in results.items():
    print(f"  {k}: {v:.2f}%")



=== Evaluating ensemble 'T12' on final_test ===
Classes: ['negative', 'positive']
Num images: 340
Num experts in ensemble: 5


final_test [T12]: 100%|██████████| 6/6 [03:35<00:00, 36.00s/it]



[T12] Overall accuracy on final_test: 92.94%

Per-class accuracy:
  Class 'negative': 98.18% (n=220)
  Class 'positive': 83.33% (n=120)

=== Evaluating ensemble 'T12r' on final_test ===
Classes: ['negative', 'positive']
Num images: 340
Num experts in ensemble: 5


final_test [T12r]: 100%|██████████| 6/6 [03:49<00:00, 38.18s/it]


[T12r] Overall accuracy on final_test: 93.24%

Per-class accuracy:
  Class 'negative': 98.64% (n=220)
  Class 'positive': 83.33% (n=120)


Summary of final accuracies (percent):
  T12: 92.94%
  T12r: 93.24%
